# 1.1. Описание проекта 

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.\

Вам предоставлены пробы нефти в трёх регионах: в каждом 100 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# 1.2. Инструкция по выполнению проекта

Загрузите и подготовьте данные. Поясните порядок действий.\
Обучите и проверьте модель для каждого региона:
- 2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
- 2.2. Обучите модель и сделайте предсказания на валидационной выборке.
- 2.3. Сохраните предсказания и правильные ответы на валидационной выборке.
- 2.4. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
- 2.5. Проанализируйте результаты.\
Подготовьтесь к расчёту прибыли:
- 3.1. Все ключевые значения для расчётов сохраните в отдельных переменных.
- 3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.
- 3.3. Напишите выводы по этапу подготовки расчёта прибыли.\
Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
- 4.1. Выберите скважины с максимальными значениями предсказаний.
- 4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
- 4.3. Рассчитайте прибыль для полученного объёма сырья.\
Посчитайте риски и прибыль для каждого региона:
- 5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
- 5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
- 5.3. Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

# 1.3. Описание данных

id — уникальный идентификатор скважины;\
f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);\
product — объём запасов в скважине (тыс. баррелей).

# 1.4. Условие задачи

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.\
Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

# 2. Загрузка и предобработка данных

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [31]:
df0 = pd.read_csv('geo_data_0.csv')
df0.head()
df1 = pd.read_csv('geo_data_1.csv')
df1.head()
df2 = pd.read_csv('geo_data_2.csv')
df2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [32]:
df = []
df.append(df0)
df.append(df1)
df.append(df2)

In [33]:
df[0] = df[0].drop(['id'], axis=1)

In [34]:
df[1] = df[1].drop(['id'], axis=1)
df[2] = df[2].drop(['id'], axis=1)

In [37]:
df

[             f0        f1        f2     product
 0      0.705745 -0.497823  1.221170  105.280062
 1      1.334711 -0.340164  4.365080   73.037750
 2      1.022732  0.151990  1.419926   85.265647
 3     -0.032172  0.139033  2.978566  168.620776
 4      1.988431  0.155413  4.751769  154.036647
 ...         ...       ...       ...         ...
 99995  0.971957  0.370953  6.075346  110.744026
 99996  1.392429 -0.382606  1.273912  122.346843
 99997  1.029585  0.018787 -1.348308   64.375443
 99998  0.998163 -0.528582  1.583869   74.040764
 99999  1.764754 -0.266417  5.722849  149.633246
 
 [100000 rows x 4 columns],
               f0         f1        f2     product
 0     -15.001348  -8.276000 -0.005876    3.179103
 1      14.272088  -3.475083  0.999183   26.953261
 2       6.263187  -5.948386  5.001160  134.766305
 3     -13.081196 -11.506057  4.999415  137.945408
 4      12.702195  -8.147433  5.004363  134.766305
 ...          ...        ...       ...         ...
 99995   9.535637  -6.878

# 3. Обучение модели и её проверка

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [64]:
df_model = []
for i in range(0,3):
    X = df[i].drop(['product'],axis=1)
    y = df[i]['product']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    prediction = linreg.predict(X_test)

    print('raw_materials_mean_stock=', df[i]['product'].mean(), '; well_mean_product_stocks=', prediction.mean(), '; RMSE=', np.sqrt(mean_squared_error(y_test, prediction)))
    df[i]['product_forecast'] = linreg.predict(df[i].drop(['product'], axis=1)) 
    df_model.append(df[i].loc[y_test.index])
    

raw_materials_mean_stock= 92.49999999999976 ; well_mean_product_stocks= 92.59285819581818 ; RMSE= 37.76023879156018
raw_materials_mean_stock= 68.82500000002561 ; well_mean_product_stocks= 68.60309883597067 ; RMSE= 0.8908254316670086
raw_materials_mean_stock= 95.00000000000041 ; well_mean_product_stocks= 94.78885362076424 ; RMSE= 40.24237823959256


Итак, мы построили модель на основе линейной регрессии для каждого из 3 регионов. \
Затем мы вычислили среднее прогноза объема запасов сырья и RMSE модели по каждому региону. \
Как можно заметить, самое низкое среднеквадратичное отклонение у 1-го региона (rmse=68,825), в сравненении с 0 и 2 регионом (92,499 и 95,000 соответственно). То есть модель предсказываем объёмы запасов сырья очень точно. \
Что касается среднего объёма месторождений в 1-ом регионе, то число значительно ниже чем в 0 и 2 (примерно на 23-26к баррелей). 

# 3. Подготовка к расчёту прибыли

In [77]:
total_budget = 10000000000
well_cost = 50000000 # косты - 50 млн, т.к. бюджет 10 млрд. на разработку 200 скважин
wells_cnt = int(total_budget/well_cost) 
oil_cost = 4500*1000 # - тысяча баррелей нефти

In [65]:
def revenue(target, forecast, count):
    probs_sorted = forecast.sort_values(ascending=False)
    selected = target[probs_sorted.index]
    return oil_cost * selected[:count].sum() - total_budget

In [67]:
print(wells_cnt) #скважин

200


In [72]:
min_avg_well_product = total_budget/(oil_cost*wells_cnt)

print("Минимально-необходимый средний объём запасов в месторождениях региона, тыс.баррелей: ",min_avg_well_product)
print("Средний объём запасов сырья в 0 регионе: ",df[0]['product'].mean())
print("Средний объём запасов сырья в 1 регионе: ",df[1]['product'].mean())
print("Средний объём запасов сырья в 2 регионе: ",df[2]['product'].mean())

Минимально-необходимый средний объём запасов в месторождениях региона, тыс.баррелей:  11.11111111111111
Средний объём запасов сырья в 0 регионе:  92.49999999999976
Средний объём запасов сырья в 1 регионе:  68.82500000002561
Средний объём запасов сырья в 2 регионе:  95.00000000000041


# 4. Расчёт прибыли

In [73]:
state = np.random.RandomState(101)

In [78]:
for i in range(0,3):  
    values = []
    count=0
    for k in range(1000):
        target_subsample = df_model[i]['product'].sample(n=500, replace=False, random_state=state) #frac=1
        forecast_subsample = df_model[i]['product_forecast'][target_subsample.index] 
        
        revenue__est = revenue(target_subsample, forecast_subsample, wells_cnt)
        values.append(revenue__est)
        
        if revenue__est<0:
            count+=1 #на случай, если у нас есть убытки
      
    
    values = pd.Series(values)
    lower = values.quantile(0.025) #95%-ый доверительный интервал
    upper = values.quantile(0.975) 
    
    mean = values.mean()
    print('Средняя прибыль по региону №',i,':', mean)
    print('Доверительный интервал: от ',lower,' до ', upper) 
    print('Вероятность убытков: ', count/1000) 

Средняя прибыль по региону № 0 : 94062852842.29729
Доверительный интервал: от  88864505954.41982  до  99158675971.16681
Вероятность убытков:  0.0
Средняя прибыль по региону № 1 : 94366404372.39455
Доверительный интервал: от  90660291379.99731  до  98465499642.62146
Вероятность убытков:  0.0
Средняя прибыль по региону № 2 : 93752992863.18561
Доверительный интервал: от  88475945255.62437  до  99163950430.52774
Вероятность убытков:  0.0


   Таким образом, при помощи метода Bootstrap мы перебрали 1000 выборок объёмом 500 месторождений, для которых затем мы вычислили прибыль (из 500 точек выбирается 200 с наилучшей оценкой запасов)
Выручка = oil_cost * selected[:count].sum() - total_budget, т.е. стоимость за 1000 баррелей умноженная на количество запасов (реальная стоимость запасов в 200 месторождениях) и минус общий бюджет на разработку скважин. 

По полученным нами 1000 оценкам прибыли вычисляется средняя, 95% доверительный интервал, а также вероятность убытков.

После проведенного исследования, мы видим, что вероятность убытков во всех регионах при разработанной модели и заданных условиях разработки - нулевая. Прибыль всегда положительная. \

  Вывод: Из 3х потенциальных регионов, полагается целесообразным выбрать регион №1, ибо он имеет наилучшую среднюю оценку прибыли - 94,366 млрд.руб. и наибольшую нижнюю границу доверительного интервала - 90,339 млрд. Очевидно, что данный результат получен вследствие значительно более точной модели, позволившей точнее предсказывать объёмы запасов по данным геологоразведки 500 месторождений и выбирать действительно более объёмные, учитывая тот факт, что средний объём месторождений в данном регионе значительно ниже, чем в аналогичных регионах 0 и 2).

